In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

这本笔记本是由[Mohammad Al-Ansari](https://github.com/Mansari)贡献的。

# GCP 上的端到端机器学习：MLOps 阶段1：数据管理：使用 Vision API 从 PDF 中创建一个未标记的 Vertex AI AutoML 文本实体提取数据集

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_visionapi_and_vertex_datasets.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_visionapi_and_vertex_datasets.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage1/get_started_with_visionapi_and_vertex_datasets.ipynb">
     <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本笔记本基于存储在云存储桶中的一组PDF文件，创建一个无标签的`Vertex AI AutoML`文本实体提取数据集。

该笔记本可进行修改，用于创建不同类型的文本数据集，包括情感分析和分类。

### 目标

在本教程中，您将学习如何使用` Vision API`从存储在云存储桶上的PDF文件中提取文本。然后，您将处理结果并创建一个适用于`AutoML`的未标记的`Vertex AI数据集`，用于文本实体提取。

然后，您可以使用Google Cloud控制台对数据集进行注释/标记，或者创建一个类似于[此笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_data_labeling.ipynb)中演示的标记作业。

本教程使用以下Google Cloud服务：

- `Vision AI`
- `Vertex AI AutoML`

执行的步骤包括：

1. 使用` Vision API`执行光学字符识别（OCR）以从PDF文件中提取文本。
2. 处理结果并将其保存到文本文件中。
3. 生成一个`Vertex AI数据集`导入文件。
4. 在`Vertex AI`中创建一个新的未标记文本实体提取`Vertex AI数据集`资源。

数据集

本教程使用的数据集是来自Google公共数据集的[带有提取结构化数据的专利PDF样本]。 

该数据集包括从美国和欧盟发出的300多份专利文件中提取的数据。 数据集包括每份专利的第一页的云存储 blob 的链接，以及一些提取的实体。

该数据作为一个[公共数据集](https://cloud.google.com/bigquery/public-data)发布在`BigQuery`上。

### 价格

本教程使用 Google Cloud 的可计费组件：

* Vision API
* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)，[Vision API 价格](https://cloud.google.com/vision/pricing)，[云存储 价格](https://cloud.google.com/storage/pricing)，并使用 [计价计算器](https://cloud.google.com/products/calculator/) 来生成基于您预期使用量的费用估算。

### 设置本地开发环境

如果您正在使用Colab或Vertex AI Workbench笔记本，您的环境已经满足运行此笔记本的所有要求。您可以跳过这一步。

否则，请确保您的环境满足此笔记本的要求。您需要以下内容：

- Vision API SDK
- Vertex AI SDK
- Cloud Storage SDK
- Git
- Python 3
- virtualenv
- 在使用Python 3运行的虚拟环境中运行Jupyter笔记本

Cloud Storage指南中提供了[设置Python开发环境](https://cloud.google.com/python/setup)的详细说明，以及[Jupyter安装指南](https://jupyter.org/install)来满足这些要求。以下步骤提供了一套简化的指导：

1. [安装并初始化SDKs](https://cloud.google.com/sdk/docs/)。

2. [安装Python 3](https://cloud.google.com/python/setup#installing_python)。

3. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

4. 要安装Jupyter，请在终端Shell中运行`pip3 install jupyter`。

5. 要启动Jupyter，请在终端Shell中运行`jupyter notebook`。

6. 在Jupyter Notebook仪表板中打开此笔记本。

## 安装

安装执行此笔记本所需的软件包。您可以忽略`pip`依赖解析器的错误，因为它们不会影响此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-storage google-cloud-vision google-cloud-aiplatform $USER_FLAG -q

### 重新启动内核

安装了额外的软件包之后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##在开始之前

###GPU运行时

*如果有此选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

###设置您的Google Cloud项目

**无论您的笔记本环境如何，都需要完成以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用用于计算/存储成本。

2. [确保为您的项目启用结算。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vision API，Vertex AI APIs，Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=vision.googleapis.com,aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在此笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`!`为前缀的行作为shell命令运行，并将以`$`为前缀的Python变量插值。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

### 地区

#### 视觉AI

您现在可以通过设置`VISION_AI_REGION`变量来指定大陆级别的数据存储和光学字符识别（OCR）处理。您可以选择以下选项之一：

* 仅美国国家：`us`
* 欧洲联盟：`eu`

了解更多关于[OCR的Vision AI区域](https://cloud.google.com/vision/docs/pdf#regionalization)

#### Vertex AI

您还可以更改`VERTEX_AI_REGION`变量，这将用于笔记本其余部分的操作。以下是Vertex AI支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能不能使用多地区存储桶来训练Vertex AI。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
VISION_AI_REGION = "[your-region]"  # @param {type: "string"}

if VISION_AI_REGION == "[your-region]":
    VISION_AI_REGION = "us"

VERTEX_AI_REGION = "[your-region]"  # @param {type: "string"}

if VERTEX_AI_REGION == "[your-region]":
    VERTEX_AI_REGION = "us-central1"

时间戳

如果你正在进行现场教程，你可能会使用一个共享的测试账户或项目。为了避免用户之间在创建资源时发生名称冲突，你为每个实例会话创建一个时间戳，并将其附加到本教程中将要创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Vertex AI数据集导入模式

这个常量告诉Vertex AI导入数据集的模式。在这个教程中，您将使用文本提取的值，但是您也可以将其更改为以下任何值，以满足其他用例：

- 
`aiplatform.schema.dataset.ioformat.text.single_label_classification`

- 
`aiplatform.schema.dataset.ioformat.text.multi_label_classification`

- 
`aiplatform.schema.dataset.ioformat.text.extraction`

- 
`aiplatform.schema.dataset.ioformat.text.sentiment`

In [ ]:
from google.cloud import aiplatform

DATASET_IMPORT_SCHEMA = aiplatform.schema.dataset.ioformat.text.extraction

### 验证您的谷歌云账户

**如果您正在使用 Vertex AI Workbench**，您的环境已经验证。跳过此步骤。如果仍然收到错误消息，您可能需要授予服务账号（您的 Workbench 笔记本正在运行的账号）访问以下列出的服务的权限。

**如果您正在使用 Colab**，运行下面的单元格，并按照提示进行身份验证，通过 oAuth 验证您的账户。

**否则**，按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务账号**。

在**服务账号名称**字段中输入一个名称，点击**创建**。

在**将此服务账号授权访问项目**部分，点击角色下拉列表。在筛选框中键入“Vertex”，并选择**Vertex AI 管理员**。在筛选框中键入“存储对象管理员”，并选择**存储对象管理员**。

点击创建。包含您密钥的 JSON 文件将下载到您的本地环境。

在下面的单元格中，输入您的服务账号密钥的路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境如何，以下步骤都是必需的。**

当您初始化 Python 版本的 Vertex AI SDK 时，您需要指定一个云存储暂存桶。这个暂存桶是您的数据集和模型资源相关数据在不同会话中保留的地方。这个桶还将用于存储 Vision API SDK 中 PDF 到文本转换过程的输出。

在下面设置您的云存储桶的名称。桶名称必须在所有谷歌云项目中是全局唯一的，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

仅当您的存储桶尚不存在时：运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $VERTEX_AI_REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查云存储桶中的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。

### 导入库并定义常量

In [ ]:
import os

from google.cloud import aiplatform, storage, vision

### 初始化Python视觉API SDK

为您的项目和地区初始化 `Vision AI` SDK for Python。

In [ ]:
vision_client_options = {
    "quota_project_id": PROJECT_ID,
    "api_endpoint": f"{VISION_AI_REGION}-vision.googleapis.com",
}
vision_client = vision.ImageAnnotatorClient(client_options=vision_client_options)

初始化Python的Vertex AI SDK，为您的项目、区域和相应的存储桶准备。

In [ ]:
aiplatform.init(
    project=PROJECT_ID, location=VERTEX_AI_REGION, staging_bucket=BUCKET_URI
)

### 初始化 Python的云存储SDK

为您的项目初始化Python的`云存储`SDK。

In [ ]:
storage_client = storage.Client(project=PROJECT_ID)

教程

现在，您可以开始从PDF文件中创建一个未标记的`Vertex AI数据集`文本实体提取数据集。

### 使用 Vision API 将 PDF 文件转换为文本

首先，您可以向 Vision API 发出请求，对存储在云存储桶中的专利样本的 PDF 进行 OCR 处理，将其转换为文本。

*注意:* Vision API 一次只允许批量提交 100 份文档。

In [ ]:
ORIGIN_BUCKET_NAME = "gcs-public-data--labeled-patents"
# You can add a path if needed
ORIGIN_BUCKET_PATH = ""

DESTINATION_BUCKET_NAME = BUCKET_NAME
DESTINATION_BUCKET_PATH = "ocr-output"

gcs_destination_uri = f"gs://{DESTINATION_BUCKET_NAME}/{DESTINATION_BUCKET_PATH}"

# Specify the feature for the Vision API processor
feature = vision.Feature(type_=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

# Retrieve a list of all files in the bucket and path
blobs = storage_client.list_blobs(
    ORIGIN_BUCKET_NAME, prefix=ORIGIN_BUCKET_PATH, delimiter="/"
)

# Create a collection of requests. The SDK requires a separate request per each
# file that we want to extract text from
async_requests = []

# Visions API only supports processing up to 100 documents at a time
# so we will process the first 100 elements only
sliced_blob_list = list(blobs)[:100]

# Loop through the source bucket and create a request for each file there
for blob in sliced_blob_list:
    # Build input_config
    # Ensure we are only processing PDF files
    if blob.name.endswith(".pdf"):
        gcs_source = vision.GcsSource(uri=f"gs://{ORIGIN_BUCKET_NAME}/{blob.name}")
        input_config = vision.InputConfig(
            gcs_source=gcs_source, mime_type="application/pdf"
        )

        # Build output config
        # Get file name
        file_name = os.path.splitext(os.path.basename(blob.name))[0]
        gcs_destination = vision.GcsDestination(
            uri=f"{gcs_destination_uri}/{file_name}-"
        )
        output_config = vision.OutputConfig(gcs_destination=gcs_destination)

        # Build request object and add to the collection
        async_request = vision.AsyncAnnotateFileRequest(
            features=[feature], input_config=input_config, output_config=output_config
        )

        async_requests.append(async_request)

print(f"Created {len(async_requests)} requests")

# Submit the batch OCR job

operation = vision_client.async_batch_annotate_files(requests=async_requests)
print("Submitting the batch OCR job")

print("Waiting for the operation to finish... this will take a short while")

response = operation.result(timeout=420)

print("Completed!")

快速查看提取的带注释的JSON文件

接下来，您可以查看一个提取的带注释的JSON文件的内容。

In [ ]:
json_files = ! gsutil ls {gcs_destination_uri}

example = json_files[0]
! gsutil cat {example} | head -n 1

### 处理结果并构建导入文件

`Vision API` 的输出是以 JSON 格式呈现，包含了详细的文本提取数据。您只需要完整的文本输出，因此您将处理 JSON 结果，提取文本输出，并保存到新的文本文件中，以便在教程中稍后使用。

In [ ]:
import json

print("Extracting text from Vision API output and saving it to text files")

ocr_blobs = storage_client.list_blobs(
    DESTINATION_BUCKET_NAME, prefix=DESTINATION_BUCKET_PATH
)

output_bucket = storage_client.bucket(DESTINATION_BUCKET_NAME)

# begin building the import file content
import_file_entries = []

for ocr_blob in ocr_blobs:
    # Only process .json files, in case we previously processed files and had .txt files
    if ocr_blob.name.endswith(".json"):
        print(f"Extracting text from {ocr_blob.name}")
        # read each blob into a stream
        contents = ocr_blob.download_as_string()
        # load as JSON
        json_object = json.loads(contents)
        # extract text
        full_text = ""
        for response in json_object["responses"]:
            if response["fullTextAnnotation"]:
                full_text += response["fullTextAnnotation"]["text"] + "\r\n"

        # save as a blob
        output_blob_name = f"{ocr_blob.name}.txt"
        import_file_blob = output_bucket.blob(output_blob_name)
        import_file_blob.upload_from_string(full_text)

        # create import file listing
        import_file_entry = {
            "textGcsUri": f"gs://{DESTINATION_BUCKET_NAME}/{output_blob_name}"
        }

        import_file_entries.append(import_file_entry)

print("Extraction completed!")

快速查看提取的文本文件

接下来，您可以查看提取的一个文本文件的内容。

In [ ]:
example = import_file_entries[0]["textGcsUri"]

! gsutil cat {example}

###生成并保存导入文件以供在“Vertex AI数据集”资源中使用

您现在将构建导入文件，该文件将用于创建“Vertex AI数据集”资源。

In [ ]:
IMPORT_FILE_PATH = "import_file"

# Convert import file entries to JSON Lines format
import_file_content = ""
for entry in import_file_entries:
    import_file_content += json.dumps(entry) + "\n"

print(f"Created import file based on {len(import_file_entries)} annotations")

# Upload content to GCS to be used in our next step
gcs_annotation_file_name = f"{IMPORT_FILE_PATH}/import_file_{TIMESTAMP}.jsonl"
import_file_blob = output_bucket.blob(gcs_annotation_file_name)
import_file_blob.upload_from_string(import_file_content)

print(f"Uploaded import file to {output_bucket.name}/{gcs_annotation_file_name}")

### 创建一个没有标签的`Vertex AI Dataset`资源

接下来，您可以使用`TextDataset`类的`create`方法创建 `Dataset` 资源，该方法需要以下参数：

- `display_name`：`Dataset` 资源的人类可读名称。
- `gcs_source`：要导入数据项到`Dataset` 资源的一个或多个数据集索引文件的列表。
- `import_schema_uri`：数据项的数据标签模式。

此操作可能需要十到二十分钟。

In [ ]:
print("Creating dataset ...")

dataset = aiplatform.TextDataset.create(
    display_name="Text Dataset " + TIMESTAMP,
    gcs_source=[f"gs://{output_bucket.name}/{gcs_annotation_file_name}"],
    import_schema_uri=DATASET_IMPORT_SCHEMA,
)

print("Completed!")

print(dataset.resource_name)

恭喜！您的数据集现在可以开始进行注释了！

您有两个选择：

* 使用谷歌云控制台，在 `Vertex AI` 中手动标注数据集。请查看[此链接](https://cloud.google.com/vertex-ai/docs/datasets/label-using-console#entity-extraction)了解详情。
* 创建一个标注工作以请求数据标注。查看[此链接](https://cloud.google.com/vertex-ai/docs/datasets/data-labeling-job)和[此笔记本](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage1/get_started_with_data_labeling.ipynb)了解更多详情和示例。

清理工作

为了清理所有在该项目中使用的GCP资源，您可以删除用于本教程的[GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
dataset.delete()

# Delete the bucket created
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI